# Setting up

## load modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import itertools as itt
import os
import sys

import holoviews as hv
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade
from holoviews.util import Dynamic
from IPython.core.display import display

## set path and parameters

In [ ]:
# Set up Initial Basic Parameters#
minian_path = "."
dpath = "./demo_movies/"
minian_ds_path = os.path.join(dpath, "minian")
intpath = "./minian_intermediate"
subset = dict(frame=slice(0, None))
subset_mc = None
interactive = True
output_size = 100
param_save_minian = {
    "dpath": minian_ds_path,
    "meta_dict": dict(session_id=-1, session=-2, animal=-3),
    "overwrite": True,
}

# Pre-processing Parameters#
param_load_videos = {
    "pattern": "msCam[0-9]+\.avi$",
    "dtype": np.uint8,
    "downsample": dict(frame=1, height=1, width=1),
    "downsample_strategy": "subset",
}
param_denoise = {"method": "median", "ksize": 7}
param_background_removal = {"method": "tophat", "wnd": 15}

# Motion Correction Parameters#
subset_mc = None
param_estimate_motion = {"dim": "frame"}

# Initialization Parameters#
param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "nchunk": 100,
    "max_wnd": 15,
    "diff_thres": 2,
}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1, "med_wnd": None}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 5, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8}

# CNMF Parameters#
param_get_noise = {"noise_range": (0.06, 0.5), "noise_method": "logmexp"}
param_first_spatial = {
    "dl_wnd": 5,
    "sparse_penal": 0.01,
    "update_background": True,
    "normalize": True,
    "size_thres": (25, None),
}
param_first_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.2,
    "zero_thres": 1e-8,
    "max_iters": 200,
    "use_smooth": True,
    "scs_fallback": False,
    "normalize": True,
    "post_scal": True,
}
param_first_merge = {"thres_corr": 0.8}
param_second_spatial = {
    "dl_wnd": 5,
    "sparse_penal": 0.01,
    "update_background": True,
    "normalize": True,
    "size_thres": (25, None),
}
param_second_temporal = {
    "noise_freq": 0.06,
    "sparse_penal": 1,
    "p": 1,
    "add_lag": 20,
    "jac_thres": 0.4,
    "zero_thres": 1e-8,
    "max_iters": 500,
    "use_smooth": True,
    "scs_fallback": False,
    "normalize": True,
    "post_scal": True,
}

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMBA_NUM_THREADS"] = "1"
os.environ["MINIAN_INTERMEDIATE"] = intpath

## import minian

In [ ]:
%%capture
sys.path.append(minian_path)
from minian.cnmf import (
    compute_AtC,
    compute_trace,
    get_noise_fft,
    smooth_sig,
    unit_merge,
    update_spatial,
    update_temporal,
)
from minian.initialization import (
    gmm_refine,
    initA,
    initbf,
    initC,
    intensity_refine,
    ks_refine,
    pnr_refine,
    seeds_init,
    seeds_merge,
)
from minian.motion_correction import estimate_motion, apply_transform
from minian.preprocessing import denoise, remove_background
from minian.utilities import load_videos, open_minian, save_minian, TaskAnnotation, get_optimal_chk
from minian.visualization import (
    CNMFViewer,
    VArrayViewer,
    generate_videos,
    visualize_motion,
    visualize_gmm_fit,
    visualize_preprocess,
    visualize_seeds,
    visualize_spatial_update,
    visualize_temporal_update,
    write_video,
)

## module initialization

In [ ]:
dpath = os.path.abspath(dpath)
hv.notebook_extension("bokeh", width=100)

## start cluster

In [ ]:
cluster = LocalCluster(n_workers=4, memory_limit="2GB", resources={'MEM': 1}, threads_per_worker=2, dashboard_address=":8787")
annt_plugin = TaskAnnotation()
cluster.scheduler.add_plugin(annt_plugin)
client = Client(cluster)

# Pre-processing

## loading videos and visualization

In [ ]:
%%time
varr = load_videos(dpath, **param_load_videos)
chk, _ = get_optimal_chk(varr, dtype=float)

In [ ]:
%%time
varr = save_minian(
    varr.chunk({'frame': chk['frame'], 'height': -1, 'width': -1}).rename("varr"),
    intpath,
    overwrite=True)

In [ ]:
hv.output(size=output_size)
if interactive:
    vaviewer = VArrayViewer(varr, framerate=5, summary=None)
    display(vaviewer.show())

## set roi for motion correction

In [ ]:
if interactive:
    try:
        subset_mc = list(vaviewer.mask.values())[0]
    except IndexError:
        pass

## subset part of video

In [ ]:
varr_ref = varr.sel(subset)

## glow removal and visualization

In [ ]:
%%time
varr_min = varr_ref.min("frame").compute()
varr_ref = varr_ref - varr_min

In [ ]:
hv.output(size=output_size)
if interactive:
    vaviewer = VArrayViewer(
        [varr.rename("original"), varr_ref.rename("glow_removed")],
        framerate=5,
        summary=None,
        layout=True,
    )
    display(vaviewer.show())

## denoise

In [ ]:
hv.output(size=output_size)
if interactive:
    display(
        visualize_preprocess(
            varr_ref.isel(frame=0).compute(),
            denoise,
            method=["median"],
            ksize=[5, 7, 9],
        )
    )

In [ ]:
varr_ref = denoise(varr_ref, **param_denoise)

## backgroun removal

In [ ]:
hv.output(size=output_size)
if interactive:
    display(
        visualize_preprocess(
            varr_ref.isel(frame=0).compute(),
            remove_background,
            method=["tophat"],
            wnd=[10, 15, 20],
        )
    )

In [ ]:
varr_ref = remove_background(varr_ref, **param_background_removal)

## save result

In [ ]:
%%time
varr_ref = save_minian(varr_ref.rename("varr_ref"), dpath=intpath, overwrite=True)

# motion correction

## estimate motion

In [ ]:
%%time
motion = estimate_motion(varr_ref.sel(subset_mc), **param_estimate_motion)

## save motion

In [ ]:
%%time
motion = save_minian(motion.rename("motion").chunk({"frame": chk['frame']}), **param_save_minian)

## visualization of motion

In [ ]:
hv.output(size=output_size)
visualize_motion(motion)

## apply transform

In [ ]:
Y = apply_transform(varr_ref, motion, fill=0)

## save result

In [ ]:
%%time
Y_fm_chk = save_minian(Y.astype(float).rename("Y_fm_chk"), intpath, overwrite=True)
Y_hw_chk = save_minian(
    Y_fm_chk.rename("Y_hw_chk"),
    intpath,
    overwrite=True,
    chunks={"frame": -1, "height": chk['height'], "width": chk['width']},
)

## visualization of motion-correction

In [ ]:
hv.output(size=output_size)
if interactive:
    vaviewer = VArrayViewer(
        [varr_ref.rename("before_mc"), Y_fm_chk.rename("after_mc")],
        framerate=5,
        summary=None,
        layout=True,
    )
    display(vaviewer.show())

In [ ]:
im_opts = dict(frame_width=500, aspect=varr_ref.sizes['width'] / varr_ref.sizes['height'], cmap="Viridis", colorbar=True)
(
    hv.Image(
        varr_ref.max("frame").compute(), ["width", "height"], label="before_mc"
    ).opts(**im_opts)
    + hv.Image(
        Y_hw_chk.max("frame").compute(), ["width", "height"], label="after_mc"
    ).opts(**im_opts)
)

## generate video for motion-correction

In [ ]:
%%time
vid_arr = xr.concat([varr_ref, Y_fm_chk], "width").chunk({"width": -1})
write_video(vid_arr, "minian_mc.mp4", dpath)

# initialization

## compute max projection

In [ ]:
max_proj = save_minian(
    Y_fm_chk.max("frame").rename("max_proj"), **param_save_minian
).compute()

## generating over-complete set of seeds

In [ ]:
%%time
seeds = seeds_init(Y_fm_chk, **param_seeds_init)

In [ ]:
hv.output(size=output_size)
visualize_seeds(max_proj, seeds)

## peak-noise-ratio refine

In [ ]:
%%time
if interactive:
    noise_freq_list = [0.005, 0.01, 0.02, 0.06, 0.1, 0.2, 0.3, 0.45, 0.6, 0.8]
    example_seeds = seeds.sample(6, axis="rows")
    example_trace = Y_hw_chk.sel(
        height=example_seeds['height'].to_xarray(),
        width=example_seeds['width'].to_xarray()).rename(**{'index': 'seed'})
    smooth_dict = dict()
    for freq in noise_freq_list:
        trace_smth_low = smooth_sig(example_trace, freq)
        trace_smth_high = smooth_sig(example_trace, freq, btype="high")
        trace_smth_low = trace_smth_low.compute()
        trace_smth_high = trace_smth_high.compute()
        hv_trace = hv.HoloMap(
            {
                "signal": (
                    hv.Dataset(trace_smth_low)
                    .to(hv.Curve, kdims=["frame"])
                    .opts(frame_width=300, aspect=2, ylabel="Signal (A.U.)")
                ),
                "noise": (
                    hv.Dataset(trace_smth_high)
                    .to(hv.Curve, kdims=["frame"])
                    .opts(frame_width=300, aspect=2, ylabel="Signal (A.U.)")
                ),
            },
            kdims="trace",
        ).collate()
        smooth_dict[freq] = hv_trace

In [ ]:
hv.output(size=output_size)
if interactive:
    hv_res = (
        hv.HoloMap(smooth_dict, kdims=["noise_freq"])
        .collate()
        .opts(aspect=2)
        .overlay("trace")
        .layout("seed")
        .cols(3)
    )
    display(hv_res)

In [ ]:
%%time
seeds, pnr, gmm = pnr_refine(Y_hw_chk, seeds, **param_pnr_refine)

In [ ]:
if gmm:
    display(visualize_gmm_fit(pnr, gmm, 100))

In [ ]:
hv.output(size=output_size)
visualize_seeds(max_proj, seeds, "mask_pnr")

## ks refine

In [ ]:
%%time
seeds = ks_refine(Y_hw_chk, seeds, **param_ks_refine)

In [ ]:
hv.output(size=output_size)
visualize_seeds(max_proj, seeds, "mask_ks")

## merge seeds

In [ ]:
%%time
seeds_final = seeds[seeds["mask_ks"] & seeds["mask_pnr"]].reset_index(drop=True)
seeds_final = seeds_merge(Y_hw_chk, max_proj, seeds_final, **param_seeds_merge)

In [ ]:
hv.output(size=output_size)
visualize_seeds(max_proj, seeds_final, "mask_mrg")

## initialize spatial matrix

In [ ]:
%%time
A_init = initA(Y_hw_chk, seeds_final[seeds_final["mask_mrg"]], **param_initialize)
A_init = save_minian(A_init.rename("A_init"), intpath, overwrite=True)

## initialize temporal matrix

In [ ]:
%%time
C_init = initC(Y_fm_chk, A_init)
C_init = save_minian(
    C_init.rename("C_init"), intpath, overwrite=True, chunks={"unit_id": 1, "frame": -1}
)

## merge units

In [ ]:
%%time
A, C = unit_merge(A_init, C_init, **param_init_merge)
A = save_minian(A.rename("A"), intpath, overwrite=True)
C = save_minian(C.rename("C"), intpath, overwrite=True)
C_chk = save_minian(
    C.rename("C_chk"), intpath, overwrite=True, chunks={"unit_id": -1, "frame": chk['frame']}
)

## initialize background terms

In [ ]:
%%time
b, f = initbf(Y_fm_chk, A, C_chk)
f = save_minian(f.rename("f"), intpath, overwrite=True)
b = save_minian(b.rename("b"), intpath, overwrite=True)

## visualization of initialization

In [ ]:
im_opts = dict(
    frame_width=500,
    aspect=A.sizes["width"] / A.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
cr_opts = dict(frame_width=750, aspect=1.5 * A.sizes["width"] / A.sizes["height"])
(
    hv.Image(A.sum("unit_id").rename("A").compute(), kdims=["width", "height"]).opts(
        **im_opts
    )
    + hv.Image(C.rename("C").compute(), kdims=["frame", "unit_id"]).opts(
        cmap="viridis", colorbar=True, **cr_opts
    )
    + hv.Image(b.rename("b").compute(), kdims=["width", "height"]).opts(**im_opts)
    + datashade(hv.Curve(f.rename("f").compute(), kdims=["frame"]), min_alpha=200).opts(
        **cr_opts
    )
).cols(2)

# CNMF

## estimate spatial noise

In [ ]:
%%time
sn_spatial = get_noise_fft(Y_hw_chk, **param_get_noise)
sn_spatial = save_minian(sn_spatial.rename("sn_spatial"), intpath, overwrite=True)

## test parameters for spatial update

In [ ]:
if interactive:
    units = np.random.choice(A.coords["unit_id"], 10, replace=False)
    units.sort()
    A_sub = A.sel(unit_id=units).persist()
    C_sub = C.sel(unit_id=units).persist()

In [ ]:
%%time
if interactive:
    sprs_ls = [0.005, 0.01, 0.05]
    A_dict = dict()
    C_dict = dict()
    for cur_sprs in sprs_ls:
        cur_A, cur_b, cur_f, cur_mask = update_spatial(
            Y_hw_chk,
            A_sub,
            b,
            C_sub,
            f,
            sn_spatial,
            in_memory=False,
            update_background=False,
            dl_wnd=param_first_spatial["dl_wnd"],
            sparse_penal=cur_sprs,
        )
        if cur_A.sizes["unit_id"]:
            A_dict[cur_sprs] = cur_A.compute()
            C_dict[cur_sprs] = C_sub.sel(unit_id=cur_mask).compute()
    hv_res = visualize_spatial_update(A_dict, C_dict, kdims=["sparse penalty"])

In [ ]:
hv.output(size=output_size)
if interactive:
    display(hv_res)

## first spatial update

In [ ]:
%%time
A_new, b_new, f_new, mask = update_spatial(
    Y_hw_chk, A, b, C, f, sn_spatial, **param_first_spatial
)

In [ ]:
hv.output(size=output_size)
opts = dict(
    plot=dict(height=A.sizes["height"], width=A.sizes["width"], colorbar=True),
    style=dict(cmap="Viridis"),
)
(
    hv.Image(A.sum("unit_id").compute().rename("A"), kdims=["width", "height"])
    .opts(**opts)
    .relabel("Spatial Footprints Initial")
    + hv.Image(
        (A.fillna(0) > 0).sum("unit_id").compute().rename("A"),
        kdims=["width", "height"],
    )
    .opts(**opts)
    .relabel("Binary Spatial Footprints Initial")
    + hv.Image(A_new.sum("unit_id").compute().rename("A"), kdims=["width", "height"])
    .opts(**opts)
    .relabel("Spatial Footprints First Update")
    + hv.Image(
        (A_new > 0).sum("unit_id").compute().rename("A"),
        kdims=["width", "height"],
    )
    .opts(**opts)
    .relabel("Binary Spatial Footprints First Update")
).cols(2)

In [ ]:
hv.output(size=output_size)
opts_im = dict(
    plot=dict(height=b.sizes["height"], width=b.sizes["width"], colorbar=True),
    style=dict(cmap="Viridis"),
)
opts_cr = dict(plot=dict(height=b.sizes["height"], width=b.sizes["height"] * 2))
(
    hv.Image(b.compute(), kdims=["width", "height"])
    .opts(**opts_im)
    .relabel("Background Spatial Initial")
    + hv.Curve(f.compute(), kdims=["frame"])
    .opts(**opts_cr)
    .relabel("Background Temporal Initial")
    + hv.Image(b_new.compute(), kdims=["width", "height"])
    .opts(**opts_im)
    .relabel("Background Spatial First Update")
    + hv.Curve(f_new.compute(), kdims=["frame"])
    .opts(**opts_cr)
    .relabel("Background Temporal First Update")
).cols(2)

## save results

In [ ]:
%%time
A = save_minian(
    A_new.rename("A"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "height": -1, "width": -1},
)
b = save_minian(b_new.rename("b"), intpath, overwrite=True)
f = save_minian(f_new.chunk({"frame": chk['frame']}).rename("f"), intpath, overwrite=True)
C = C.sel(unit_id=A.coords["unit_id"].values)
C_chk = C_chk.sel(unit_id=A.coords["unit_id"].values)

## test parameters for temporal update

In [ ]:
if interactive:
    units = np.random.choice(A.coords["unit_id"], 10, replace=False)
    units.sort()
    A_sub = A.sel(unit_id=units).persist()
    C_sub = C_chk.sel(unit_id=units).persist()

In [ ]:
%%time
if interactive:
    p_ls = [1]
    sprs_ls = [0.1, 0.5, 1, 2]
    add_ls = [20]
    noise_ls = [0.06]
    YA_dict, C_dict, S_dict, g_dict, sig_dict, A_dict = [dict() for _ in range(6)]
    YrA = (
        compute_trace(Y_fm_chk, A_sub, b, C_sub, f)
        .persist()
        .chunk({"unit_id": 1, "frame": -1})
    )
    for cur_p, cur_sprs, cur_add, cur_noise in itt.product(
        p_ls, sprs_ls, add_ls, noise_ls
    ):
        ks = (cur_p, cur_sprs, cur_add, cur_noise)
        print(
            "p:{}, sparse penalty:{}, additional lag:{}, noise frequency:{}".format(
                cur_p, cur_sprs, cur_add, cur_noise
            )
        )
        cur_C, cur_S, cur_b0, cur_c0, cur_g, cur_mask = update_temporal(
            A_sub,
            C_sub,
            YrA=YrA,
            sparse_penal=cur_sprs,
            p=cur_p,
            use_smooth=True,
            add_lag=cur_add,
            noise_freq=cur_noise,
        )
        YA_dict[ks], C_dict[ks], S_dict[ks], g_dict[ks], sig_dict[ks], A_dict[ks] = (
            YrA.compute(),
            cur_C.compute(),
            cur_S.compute(),
            cur_g.compute(),
            (cur_C + cur_b0 + cur_c0).compute(),
            A_sub.compute(),
        )
    hv_res = visualize_temporal_update(
        YA_dict,
        C_dict,
        S_dict,
        g_dict,
        sig_dict,
        A_dict,
        kdims=["p", "sparse penalty", "additional lag", "noise frequency"],
    )

In [ ]:
hv.output(size=output_size)
if interactive:
    display(hv_res)

## first temporal update

In [ ]:
%%time
YrA = save_minian(
    compute_trace(Y_fm_chk, A, b, C_chk, f).rename("YrA"), intpath, overwrite=True,
    chunks={'unit_id': 1, 'frame': -1}
)

In [ ]:
%%time
C_new, S_new, b0_new, c0_new, g, mask = update_temporal(
    A, C, YrA=YrA, **param_first_temporal
)

In [ ]:
hv.output(size=output_size)
opts_im = dict(frame_width=500, aspect=2, colorbar=True, cmap="Viridis")
(
    hv.Image(C.compute().rename("ci"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Temporal Trace Initial")
    + hv.Div("")
    + hv.Image(C_new.compute().rename("c1"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Temporal Trace First Update")
    + hv.Image(S_new.compute().rename("s1"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Spikes First Update")
).cols(2)

In [ ]:
hv.output(size=output_size)
if interactive:
    h, w = A.sizes["height"], A.sizes["width"]
    im_opts = dict(aspect=w / h, frame_width=500, cmap="Viridis")
    cr_opts = dict(aspect=3, frame_width=1000)
    bad_units = mask.where(mask == False, drop=True).coords["unit_id"].values
    if len(bad_units) > 0:
        hv_res = (
            hv.NdLayout(
                {
                    "Spatial Footprint": Dynamic(
                        hv.Dataset(A.sel(unit_id=bad_units).compute().rename("A"))
                        .to(hv.Image, kdims=["width", "height"])
                        .opts(**im_opts)
                    ),
                    "Spatial Footprints of Accepted Units": Dynamic(
                        hv.Image(
                            A.sel(unit_id=mask).sum("unit_id").compute().rename("A"),
                            kdims=["width", "height"],
                        ).opts(**im_opts)
                    ),
                }
            )
            + datashade(
                hv.Dataset(YrA.sel(unit_id=bad_units).rename("raw")).to(
                    hv.Curve, kdims=["frame"]
                )
            )
            .opts(**cr_opts)
            .relabel("Temporal Trace")
        ).cols(1)
        display(hv_res)
    else:
        print("No rejected units to display")

In [ ]:
hv.output(size=output_size)
if interactive:
    sig = C_new + b0_new + c0_new
    display(
        visualize_temporal_update(
            YrA.sel(unit_id=mask),
            C_new,
            S_new,
            g,
            sig,
            A.sel(unit_id=mask),
        )
    )

## save results

In [ ]:
%%time
C = save_minian(
    C_new.rename("C").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
C_chk = save_minian(
    C.rename("C_chk"), intpath, overwrite=True, chunks={"unit_id": -1, "frame": chk['frame']}
)
S = save_minian(
    S_new.rename("S").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
b0 = save_minian(
    b0_new.rename("b0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
c0 = save_minian(
    c0_new.rename("c0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
A = A.sel(unit_id=C.coords["unit_id"].values)

## merge units

In [ ]:
%%time
A_mrg, C_mrg, [sig_mrg] = unit_merge(A, C, [C + b0 + c0], **param_first_merge)

In [ ]:
hv.output(size=output_size)
opts_im = dict(frame_width=500, aspect=2, colorbar=True, cmap="Viridis")
(
    hv.Image(C.compute().rename("c1"), kdims=["frame", "unit_id"])
    .relabel("Temporal Signals Before Merge")
    .opts(**opts_im)
    + hv.Image(C_mrg.compute().rename("c2"), kdims=["frame", "unit_id"])
    .relabel("Temporal Signals After Merge")
    .opts(**opts_im)
)

In [ ]:
%%time
A = save_minian(A_mrg.rename("A_mrg"), intpath, overwrite=True)
C = save_minian(C_mrg.rename("C_mrg"), intpath, overwrite=True)
C_chk = save_minian(
    C.rename("C_mrg_chk"), intpath, overwrite=True, chunks={"unit_id": -1, "frame": chk['frame']}
)
sig = save_minian(sig_mrg.rename("sig_mrg"), intpath, overwrite=True)

## test parameters for spatial update

In [ ]:
if interactive:
    units = np.random.choice(A.coords["unit_id"], 10, replace=False)
    units.sort()
    A_sub = A.sel(unit_id=units).persist()
    C_sub = sig.sel(unit_id=units).persist()

In [ ]:
%%time
if interactive:
    sprs_ls = [5e-3, 1e-2, 5e-2]
    A_dict = dict()
    C_dict = dict()
    for cur_sprs in sprs_ls:
        cur_A, cur_b, cur_f, cur_mask = update_spatial(
            Y_hw_chk,
            A_sub,
            b,
            C_sub,
            f,
            sn_spatial,
            in_memory=False,
            update_background=False,
            dl_wnd=param_second_spatial["dl_wnd"],
            sparse_penal=cur_sprs,
        )
        if cur_A.sizes["unit_id"]:
            A_dict[cur_sprs] = cur_A.compute()
            C_dict[cur_sprs] = C_sub.sel(unit_id=cur_mask).compute()
    hv_res = visualize_spatial_update(A_dict, C_dict, kdims=["sparse penalty"])

In [ ]:
hv.output(size=output_size)
if interactive:
    display(hv_res)

## second spatial update

In [ ]:
%%time
A_new, b_new, f_new, mask = update_spatial(
    Y_hw_chk, A, b, sig, f, sn_spatial, **param_second_spatial
)

In [ ]:
hv.output(size=output_size)
opts = dict(
    plot=dict(height=A.sizes["height"], width=A.sizes["width"], colorbar=True),
    style=dict(cmap="Viridis"),
)
(
    hv.Image(A.sum("unit_id").compute().rename("A"), kdims=["width", "height"])
    .opts(**opts)
    .relabel("Spatial Footprints Last")
    + hv.Image(
        (A.fillna(0) > 0).sum("unit_id").compute().rename("A"),
        kdims=["width", "height"],
    )
    .opts(**opts)
    .relabel("Binary Spatial Footprints Last")
    + hv.Image(A_new.sum("unit_id").compute().rename("A"), kdims=["width", "height"])
    .opts(**opts)
    .relabel("Spatial Footprints New")
    + hv.Image(
        (A_new > 0).sum("unit_id").compute().rename("A"),
        kdims=["width", "height"],
    )
    .opts(**opts)
    .relabel("Binary Spatial Footprints New")
).cols(2)

In [ ]:
hv.output(size=output_size)
opts_im = dict(
    plot=dict(height=b.sizes["height"], width=b.sizes["width"], colorbar=True),
    style=dict(cmap="Viridis"),
)
opts_cr = dict(plot=dict(height=b.sizes["height"], width=b.sizes["height"] * 2))
(
    hv.Image(b.compute(), kdims=["width", "height"])
    .opts(**opts_im)
    .relabel("Background Spatial Last")
    + hv.Curve(f.compute(), kdims=["frame"])
    .opts(**opts_cr)
    .relabel("Background Temporal Last")
    + hv.Image(b_new.compute(), kdims=["width", "height"])
    .opts(**opts_im)
    .relabel("Background Spatial New")
    + hv.Curve(f_new.compute(), kdims=["frame"])
    .opts(**opts_cr)
    .relabel("Background Temporal New")
).cols(2)

## save results

In [ ]:
%%time
A = save_minian(
    A_new.rename("A"),
    intpath,
    overwrite=True,
    chunks={"unit_id": 1, "height": -1, "width": -1},
)
b = save_minian(b_new.rename("b"), intpath, overwrite=True)
f = save_minian(f_new.chunk({"frame": chk['frame']}).rename("f"), intpath, overwrite=True)
C = C.sel(unit_id=A.coords["unit_id"].values)
C_chk = C_chk.sel(unit_id=A.coords["unit_id"].values)

## test parameters for temporal update

In [ ]:
if interactive:
    units = np.random.choice(A.coords["unit_id"], 10, replace=False)
    units.sort()
    A_sub = A.sel(unit_id=units).persist()
    C_sub = C_chk.sel(unit_id=units).persist()

In [ ]:
%%time
if interactive:
    p_ls = [1]
    sprs_ls = [0.1, 0.5, 1, 2]
    add_ls = [20]
    noise_ls = [0.06]
    YA_dict, C_dict, S_dict, g_dict, sig_dict, A_dict = [dict() for _ in range(6)]
    YrA = (
        compute_trace(Y_fm_chk, A_sub, b, C_sub, f)
        .persist()
        .chunk({"unit_id": 1, "frame": -1})
    )
    for cur_p, cur_sprs, cur_add, cur_noise in itt.product(
        p_ls, sprs_ls, add_ls, noise_ls
    ):
        ks = (cur_p, cur_sprs, cur_add, cur_noise)
        print(
            "p:{}, sparse penalty:{}, additional lag:{}, noise frequency:{}".format(
                cur_p, cur_sprs, cur_add, cur_noise
            )
        )
        cur_C, cur_S, cur_b0, cur_c0, cur_g, cur_mask = update_temporal(
            A_sub,
            C_sub,
            YrA=YrA,
            sparse_penal=cur_sprs,
            p=cur_p,
            use_smooth=True,
            add_lag=cur_add,
            noise_freq=cur_noise,
        )
        YA_dict[ks], C_dict[ks], S_dict[ks], g_dict[ks], sig_dict[ks], A_dict[ks] = (
            YrA.compute(),
            cur_C.compute(),
            cur_S.compute(),
            cur_g.compute(),
            (cur_C + cur_b0 + cur_c0).compute(),
            A_sub.compute(),
        )
    hv_res = visualize_temporal_update(
        YA_dict,
        C_dict,
        S_dict,
        g_dict,
        sig_dict,
        A_dict,
        kdims=["p", "sparse penalty", "additional lag", "noise frequency"],
    )

In [ ]:
hv.output(size=output_size)
if interactive:
    display(hv_res)

## second temporal update

In [ ]:
%%time
YrA = save_minian(
    compute_trace(Y_fm_chk, A, b, C_chk, f).rename("YrA"), intpath, overwrite=True,
    chunks={'unit_id': 1, 'frame': -1}
)

In [ ]:
%%time
C_new, S_new, b0_new, c0_new, g, mask = update_temporal(
    A, C, YrA=YrA, **param_second_temporal
)

In [ ]:
hv.output(size=output_size)
opts_im = dict(frame_width=500, aspect=2, colorbar=True, cmap="Viridis")
(
    hv.Image(C.compute().rename("c1"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Temporal Trace Last")
    + hv.Image(S.compute().rename("s1"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Spikes Last")
    + hv.Image(C_new.compute().rename("c2"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Temporal Trace New")
    + hv.Image(S_new.compute().rename("s2"), kdims=["frame", "unit_id"])
    .opts(**opts_im)
    .relabel("Spikes New")
).cols(2)

In [ ]:
hv.output(size=output_size)
if interactive:
    h, w = A.sizes["height"], A.sizes["width"]
    im_opts = dict(aspect=w / h, frame_width=500, cmap="Viridis")
    cr_opts = dict(aspect=3, frame_width=1000)
    bad_units = mask.where(mask == False, drop=True).coords["unit_id"].values
    if len(bad_units) > 0:
        hv_res = (
            hv.NdLayout(
                {
                    "Spatial Footprint": Dynamic(
                        hv.Dataset(A.sel(unit_id=bad_units).compute().rename("A"))
                        .to(hv.Image, kdims=["width", "height"])
                        .opts(**im_opts)
                    ),
                    "Spatial Footprints of Accepted Units": Dynamic(
                        hv.Image(
                            A.sel(unit_id=mask).sum("unit_id").compute().rename("A"),
                            kdims=["width", "height"],
                        ).opts(**im_opts)
                    ),
                }
            )
            + datashade(
                hv.Dataset(YrA.sel(unit_id=bad_units).rename("raw")).to(
                    hv.Curve, kdims=["frame"]
                )
            )
            .opts(**cr_opts)
            .relabel("Temporal Trace")
        ).cols(1)
        display(hv_res)
    else:
        print("No rejected units to display")

In [ ]:
hv.output(size=output_size)
if interactive:
    sig = C_new + b0_new + c0_new
    display(
        visualize_temporal_update(
            YrA.sel(unit_id=mask),
            C_new,
            S_new,
            g,
            sig,
            A.sel(unit_id=mask),
        )
    )

## save results

In [ ]:
%%time
C = save_minian(
    C_new.rename("C").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
C_chk = save_minian(
    C.rename("C_chk"), intpath, overwrite=True, chunks={"unit_id": -1, "frame": chk['frame']}
)
S = save_minian(
    S_new.rename("S").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
b0 = save_minian(
    b0_new.rename("b0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
c0 = save_minian(
    c0_new.rename("c0").chunk({"unit_id": 1, "frame": -1}), intpath, overwrite=True
)
A = A.sel(unit_id=C.coords["unit_id"].values)

## visualization

In [ ]:
%%time
generate_videos(varr, Y_fm_chk, A=A, C=C_chk, vpath=dpath)

In [ ]:
%%time
if interactive:
    cnmfviewer = CNMFViewer(A=A, C=C, S=S, org=Y_fm_chk)

In [ ]:
hv.output(size=output_size)
if interactive:
    display(cnmfviewer.show())

## save unit labels

In [ ]:
if interactive:
    A = A.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    C = C.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    S = S.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    c0 = c0.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))
    b0 = b0.assign_coords(unit_labels=("unit_id", cnmfviewer.unit_labels))

## save final results

In [ ]:
%%time
A = save_minian(A.rename("A"), **param_save_minian)
C = save_minian(C.rename("C"), **param_save_minian)
S = save_minian(S.rename("S"), **param_save_minian)
c0 = save_minian(c0.rename("c0"), **param_save_minian)
b0 = save_minian(b0.rename("b0"), **param_save_minian)
b = save_minian(b.rename("b"), **param_save_minian)
f = save_minian(f.rename("f"), **param_save_minian)

## close cluster

In [ ]:
client.close()
cluster.close()